In [2]:
import numpy as np
import QPP_Funcs as qpp

data1 = "/Users/chris/Documents/QPP/SolarFlareGPs/data/120704187_ctime_lc.txt"
t, I = np.loadtxt(data1, unpack=True)
Ierr = np.sqrt(I)
loc = "Data/Real/"
label = "120704187"
qpp.analyze_flare(loc, t, I, qpolabel=label)


/Users/chris/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


OSError: /Users/chris/Documents/QPP/SolarFlareGPs/data/120704187_ctime_lc.txt not found.

In [1]:
%matplotlib qt
import celerite as ce
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal
fname = loc + label
head1, t, I, optparams1, chain1, res1 = qpp.load_flare(fname+"/data1")
p1 = np.array(chain1)[-1:]

head2, t, I, optparams2, chain2, res2 = qpp.load_flare(fname+"/data2")
p2 = np.array(chain2)[-1:]

print ('Params for QPO: ' + str(p1[:3]))
print ('Params for RedNoise: ' + str(p2[:2]))


k1 = qpp.SHOTerm_Prior(log_S0 = p1[0], log_Q = p1[1], log_omega0 = p1[2])
m1 = qpp.CTSModel_prior(log_A = p1[3], log_tau1 = p1[4], log_tau2 = p1[5], log_bkg = p1[6])
gp1 = ce.GP(k1, mean = m1, fit_mean = True)

k2 = qpp.RealTerm_Prior(log_a = p2[0], log_c = p2[1])
m2 = qpp.CTSModel_prior(log_A = p2[2], log_tau1 = p2[3], log_tau2 = p2[4], log_bkg = p2[5])
gp2 = ce.GP(k2, mean = m2, fit_mean = True)

gp1.compute(t, yerr = np.sqrt(I))
gp2.compute(t, yerr = np.sqrt(I))

cov1 = gp1.get_matrix()
cov2 = gp2.get_matrix()

I1, var1 = gp1.predict(I, t, return_var=True)
I2, var2 = gp2.predict(I, t, return_var=True)

plt.errorbar(t, I, yerr = np.sqrt(I), fmt='k.', label = 'real')
plt.plot(t, I1, 'm--', alpha = 1, label = 'qpo-only sample')
plt.plot(t, I2, 'c--', alpha = 1, label = 'rednoise sample')

plt.fill_between(t, I1+np.sqrt(var1), I1-np.sqrt(var1), color='m', alpha=0.3, edgecolor="none")
plt.fill_between(t, I2+np.sqrt(var2), I2-np.sqrt(var2), color='c', alpha=0.3, edgecolor="none")

'''
for i in range(5):
    I1 = gp1.sample_conditional(I)
    I2 = gp2.sample_conditional(I)
    plt.plot(t, I1, 'm--', alpha = .5)
    plt.plot(t, I2, 'c--', alpha = .5)
'''
plt.legend()
plt.show()





NameError: name 'loc' is not defined